<a href="https://colab.research.google.com/github/JonNData/naive_bayes/blob/master/notebooks/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Naive Bayes
Multinomial 
Need a class.  
Fit and predict methods.  


*   Fit will get the histogram frequency, and train the model, update attributes.
-- Need a feature vector, row is obs, columns are features, numerical
*   Predict will classify the observation  

  

What's the input? Needs to be categorical but that can be easily arranged.


## Quick Example with Balance Scale data

In [8]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
headers = ['class_name', 'left-weight', 'left-distance', 'right-weight', 'right-distance']

In [3]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/balance-scale/balance-scale.data', names=headers)
df

,class_name,left-weight,left-distance,right-weight,right-distance
0,B,1,1,1,1
1,R,1,1,1,2
2,R,1,1,1,3
3,R,1,1,1,4
4,R,1,1,1,5
...,...,...,...,...,...
620,L,5,5,5,1
621,L,5,5,5,2
622,L,5,5,5,3
623,L,5,5,5,4


In [4]:
train, test = train_test_split(df, test_size = 0.2)

In [5]:
X_train = train.drop(columns='class_name')
X_test = test.drop(columns='class_name')

y_train = train['class_name'].apply(lambda x: 0 if x=="L" else 1) 
y_test = test['class_name'].apply(lambda x: 0 if x=="L" else 1)

In [7]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
y_pred = mnb.predict(X_test)
accuracy_score(y_test, y_pred=y_pred)

0.968

## Now to make the class


In [10]:
import numpy as np

In [39]:
class GaussBayes:
  def __init__(self):
    pass
  def fit(self, X_train, y_train):
    # Get information about the array
    num_obs, num_features = X_train.shape
    
    # Get array of unique targets
    self.u_classes = np.unique(y_train)
    num_classes = len(self.u_classes)

    # intialize mean, variance, priors with correct size
    self.X_train_mean = np.zeros((num_classes, num_features), dtype=np.float64)
    self.var = np.zeros((num_classes, num_features), dtype=np.float64)
    self.priors = np.zeros(num_classes, dtype=np.float64)

    # Calculate
    for i,cls in enumerate(self.u_classes):
      # specify only values where the class is the same as the target
      X_train_cls = X_train[cls==y_train]
      # the mean from this class and all columns
      self.X_train_mean[cls,:] = X_train_cls.mean(axis=0)
      self.var[cls,:] = X_train_cls.var(axis=0)
      self.priors[cls] = X_train_cls.shape[0] / float(num_obs) # how often this class is occuring

  def predict(self, X_test):
    y_pred = [self._predict(x) for x in X_test]
    return y_pred

  def _predict(self, X_test):
    posteriors =  []

    for i, cls in enumerate(self.u_classes):
      prior = np.log(self.priors[i])
      class_conditional = np.sum(np.log(self._prob_density(i, X_test)))
      posterior =  prior + class_conditional
      posteriors.append(posterior)
    
    return self.u_classes[np.argmax(posteriors)] # argmax gives index of max value

  def _prob_density(self, class_idx, x):
      mean = self.X_train_mean[class_idx]
      var = self.var[class_idx]
      numerator = np.exp(- (x-mean)**2 / (2 * var))
      denominator = np.sqrt(2 * np.pi * var)
      return numerator / denominator

In [40]:
gb = GaussBayes()
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
accuracy_score(y_test, y_pred=y_pred)

UFuncTypeError: ignored

In [43]:
class NaiveBayes:
    def __init__(self, X, y):
        self.num_examples, self.num_features = X.shape
        self.num_classes = len(np.unique(y))
        self.eps = 1e-6

    def fit(self, X, y):
        self.classes_mean = {}
        self.classes_variance = {}
        self.classes_prior = {}

        for c in range(self.num_classes):
            X_c = X[y == c]

            self.classes_mean[str(c)] = np.mean(X_c, axis=0)
            self.classes_variance[str(c)] = np.var(X_c, axis=0)
            self.classes_prior[str(c)] = X_c.shape[0] / X.shape[0]

    def predict(self, X):
        probs = np.zeros((self.num_examples, self.num_classes))

        for c in range(self.num_classes):
            prior = self.classes_prior[str(c)]
            probs_c = self.density_function(
                X, self.classes_mean[str(c)], self.classes_variance[str(c)]
            )
            probs[:, c] = probs_c + np.log(prior)

        return np.argmax(probs, 1)

    def density_function(self, x, mean, sigma):
        # Calculate probability from Gaussian density function
        const = -self.num_features / 2 * np.log(2 * np.pi) - 0.5 * np.sum(
            np.log(sigma + self.eps)
        )
        probs = 0.5 * np.sum(np.power(x - mean, 2) / (sigma + self.eps), 1)
        return const - probs


In [ ]:
nb = NaiveBayes(X_train, y_train)
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
accuracy_score(y_test, y_pred=y_pred)